#1

In [ ]:
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def peek(self):
        return self.items[-1]

    def size(self):
        return len(self.items)


class BinaryTree:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild == None:
            self.leftChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t

    def insertRight(self, newNode):
        if self.rightChild == None:
            self.rightChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t

    def getRightChild(self):
        return self.rightChild

    def getLeftChild(self):
        return self.leftChild

    def setRootVal(self, obj):
        self.key = obj

    def getRootVal(self):
        return self.key


def buildParseTree(fpexp):
    fplist = fpexp.split()
    pStack = Stack()
    eTree = BinaryTree('')
    pStack.push(eTree)
    currentTree = eTree
    for i in fplist:
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i not in ['or', 'and', 'not', ')']:
            currentTree.setRootVal(i == 'true')
            parent = pStack.pop()
            currentTree = parent
        elif i in ['or', 'and']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()
        elif i == 'not':
            currentTree.setRootVal(i)
            currentTree.insertLeft('')
            pStack.push(currentTree)
            currentTree = currentTree.getLeftChild()
        elif i == ')':
            currentTree = pStack.pop()
        else:
            raise ValueError
    return eTree


pt = buildParseTree("( ( false or false ) and true )")

In [ ]:
def evaluate(parseTree):
    opers = {'or': lambda x, y: x or y, 'and': lambda x, y: x and y, 'not': lambda x: not x}

    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()

    if leftC and rightC:
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC),evaluate(rightC))
    elif leftC:
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC))
    else:
        return parseTree.getRootVal()

In [ ]:
print(evaluate(pt))

False


In [ ]:
def printexp(parseTree):
    if parseTree != None:
        if parseTree.getLeftChild():
            print('(', end='')
            printexp(parseTree.getLeftChild())
            print(')', end='')
        print(parseTree.getRootVal(), end='')
        if parseTree.getRightChild():
            print('(', end='')
            printexp(parseTree.getRightChild())
            print(')', end='')

printexp(pt)

((False)or(False))and(True)

#2

In [ ]:
class TreeNode:
    def __init__(self, key, val, left=None, right=None, parent=None):
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent

    def hasLeftChild(self):
        return self.leftChild

    def hasRightChild(self):
        return self.rightChild

    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self

    def isRightChild(self):
        return self.parent and self.parent.rightChild == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.rightChild or self.leftChild)

    def hasAnyChildren(self):
        return self.rightChild or self.leftChild

    def hasBothChildren(self):
        return self.rightChild and self.leftChild

    def replaceNodeData(self, key, value, lc, rc):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self

    def setPayload(self, value):
        self.payload = value


class BinarySearchTree:

    def __init__(self):
        self.root = None
        self.size = 0

    def length(self):
        return self.size

    def __len__(self):
        return self.size

    def put(self, key, val):
        if self.root:
            self._put(key, val, self.root)
        else:
            self.root = TreeNode(key, val)
        self.size = self.size + 1

    def _put(self, key, val, currentNode):
        if key == currentNode.key:
            currentNode.setPayload(val)
        elif key < currentNode.key:
            if currentNode.hasLeftChild():
                self._put(key, val, currentNode.leftChild)
            else:
                currentNode.leftChild = TreeNode(key, val, parent=currentNode)
        else:
            if currentNode.hasRightChild():
                self._put(key, val, currentNode.rightChild)
            else:
                currentNode.rightChild = TreeNode(key, val, parent=currentNode)

    def __setitem__(self, k, v):
        self.put(k, v)

    def get(self, key):
        if self.root:
            res = self._get(key, self.root)
            if res:
                return res.payload
            else:
                return None
        else:
            return None

    def _get(self, key, currentNode):
        if not currentNode:
            return None
        elif currentNode.key == key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key, currentNode.leftChild)
        else:
            return self._get(key, currentNode.rightChild)

    def __getitem__(self, key):
        return self.get(key)

    def __contains__(self, key):
        if self._get(key, self.root):
            return True
        else:
            return False

    def delete(self, key):
        if self.size > 1:
            nodeToRemove = self._get(key, self.root)
            if nodeToRemove:
                self.remove(nodeToRemove)
                self.size = self.size - 1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    def __delitem__(self, key):
        self.delete(key)

    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.hasAnyChildren():
            if self.hasLeftChild():
                if self.isLeftChild():
                    self.parent.leftChild = self.leftChild
                else:
                    self.parent.rightChild = self.leftChild
                self.leftChild.parent = self.parent
            else:
                if self.isLeftChild():
                    self.parent.leftChild = self.rightChild
                else:
                    self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent

    def findSuccessor(self):
        succ = None
        if self.hasRightChild():
            succ = self.rightChild.findMin()
        else:
            if self.parent:
                if self.isLeftChild():
                    succ = self.parent
                else:
                    self.parent.rightChild = None
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ

    def findMin(self):
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current

    def remove(self, currentNode):
        if currentNode.isLeaf():  # leaf
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        elif currentNode.hasBothChildren():  # interior
            succ = currentNode.findSuccessor()
            succ.spliceOut()
            currentNode.key = succ.key
            currentNode.payload = succ.payload

        else:  # this node has one child
            if currentNode.hasLeftChild():
                if currentNode.isLeftChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild
                elif currentNode.isRightChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild
                else:
                    currentNode.replaceNodeData(currentNode.leftChild.key,
                                                currentNode.leftChild.payload,
                                                currentNode.leftChild.leftChild,
                                                currentNode.leftChild.rightChild)
            else:
                if currentNode.isLeftChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild
                elif currentNode.isRightChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                else:
                    currentNode.replaceNodeData(currentNode.rightChild.key,
                                                currentNode.rightChild.payload,
                                                currentNode.rightChild.leftChild,
                                                currentNode.rightChild.rightChild)


mytree = BinarySearchTree()
mytree[3] = "red"
mytree[4] = "blue"
mytree[6] = "yellow"
mytree[2] = "at"
print(mytree[2])
mytree[2] = "fro"
print(mytree[2])

print(mytree[6])
print(mytree[2])


at
fro
yellow
fro


In [ ]:
def has_no_duplicates(tree):
    if tree != None:
        arr = []
        traverseNode(tree.root, arr)
        dups = [i for i in set(arr) if arr.count(i) > 1]
        return len(dups) == 0
    return True


def traverseNode(node, arr):
    if node != None:
        if node.hasLeftChild():
            traverseNode(node.leftChild, arr)
        arr.append(node.key)
        if node.hasRightChild():
            traverseNode(node.rightChild, arr)

has_no_duplicates(mytree)

True

#3

In [ ]:
def animals():
    tree = BinaryTree("Is it a mammal?")
    tree.insertLeft('Cat')
    tree.insertRight('Snake')

    while True:
        playAnimals(tree)

def insertNewAnswer(node):
    old_anim = node.getRootVal()
    print("What animal do you have in mind?")
    anim = input()
    print("Enter a clarifying question.")
    question = input()
    print(f"What is the answer for {anim}?")
    resp = input().lower()
    node.setRootVal(question)
    if resp == "yes":
        node.insertLeft(anim)
        node.insertRight(old_anim)
    else:
        node.insertLeft(old_anim)
        node.insertRight(anim)

def playAnimals(node):
    if node.getLeftChild() == None:
        print(f"Is it a {node.getRootVal()}?")
        resp = input().lower()
        if resp == "yes":
            print("I won!")
            return
        print("I give up!")
        insertNewAnswer(node)
        return

    print(node.getRootVal())
    resp = input().lower()
    if resp == "yes":
        playAnimals(node.getLeftChild())
    else:
        playAnimals(node.getRightChild())

animals()

Is it a mammal?
Yes
Is it a Cat?
no
I give up!
What animal do you have in mind?
Dog
Enter a clarifying question.
Does it bark?
What is the answer for Dog?
yes
Is it a mammal?
yes
Does it bark?
yes
Is it a Dog?
yes
I won!
Is it a mammal?
no
Is it a Snake?
no
I give up!
What animal do you have in mind?
Bird
Enter a clarifying question.
Is it covered in scales?
What is the answer for Bird?
no
Is it a mammal?
no
Is it covered in scales?
no
Is it a Bird?
yes
I won!
Is it a mammal?


KeyboardInterrupt: Interrupted by user

#4

In [ ]:
class BinHeap:
    def __init__(self, limit):
        self.heapList = [0]
        self.currentSize = 0
        self.limit = limit

    def percUp(self, i):
        while i // 2 > 0:
            if self.heapList[i] < self.heapList[i // 2]:
                tmp = self.heapList[i // 2]
                self.heapList[i // 2] = self.heapList[i]
                self.heapList[i] = tmp
            i = i // 2

    def insert(self, k):
        self.heapList.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)
        if self.currentSize > self.limit:
            self.removeMax()

    def percDown(self, i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapList[i] > self.heapList[mc]:
                tmp = self.heapList[i]
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            i = mc

    def minChild(self, i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i * 2] < self.heapList[i * 2 + 1]:
                return i * 2
            else:
                return i * 2 + 1

    def removeMax(self):
        n = self.currentSize
        max = self.heapList[n // 2]
        for i in range(n//2 + 1, n):
            max = max(max, self.heapList[i])
        i = n // 2 + 1
        while self.heapList[i] != max: i += 1
        self.heapList
        for j in range(i + 1, n)


    def delMin(self):
        retval = self.heapList[1]
        self.heapList[1] = self.heapList[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapList.pop()
        self.percDown(1)
        return retval

    def buildHeap(self, alist):
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1



SyntaxError: expected ':' (ipython-input-3644861528.py, line 45)

In [ ]:
myheap = BinHeap(6)
myheap.buildHeap([1, 3, 5, 2, 10])
myheap.insert(4)
myheap.insert(7)
myheap.insert(8)
myheap.insert(1)
while myheap.currentSize > 0:
    print(myheap.delMin(), end=' ')

3 4 5 7 8 10 

#5

In [ ]:
class MaxBinHeap:
    def __init__(self):
        self.heapList = [0]
        self.currentSize = 0

    def percUp(self, i):
        while i // 2 > 0:
            if self.heapList[i] > self.heapList[i // 2]:
                tmp = self.heapList[i // 2]
                self.heapList[i // 2] = self.heapList[i]
                self.heapList[i] = tmp
            i = i // 2

    def insert(self, k):
        self.heapList.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)

    def percDown(self, i):
        while (i * 2) <= self.currentSize:
            mc = self.maxChild(i)
            if self.heapList[i] < self.heapList[mc]:
                tmp = self.heapList[i]
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            i = mc

    def maxChild(self, i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i * 2] > self.heapList[i * 2 + 1]:
                return i * 2
            else:
                return i * 2 + 1

    def delMax(self):
        retval = self.heapList[1]
        self.heapList[1] = self.heapList[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapList.pop()
        self.percDown(1)
        return retval

    def buildHeap(self, alist):
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1

In [ ]:
myheap = MaxBinHeap()
myheap.buildHeap([3, 5, 2, 10])
myheap.insert(4)
myheap.insert(7)
myheap.insert(8)
myheap.insert(1)
while myheap.currentSize > 0:
    print(myheap.delMax(), end=' ')

10 8 7 5 4 3 2 1 

#6

In [ ]:
class PriorityQueue:
    def __init__(self):
        self.heap = MaxBinHeap()
        self.translation = {}

    def enqueue(self, val, priority):
        self.heap.insert(priority)
        self.translation[priority] = val

    def dequeue(self):
        return self.translation[self.heap.delMax()]


In [ ]:
queue = PriorityQueue()

queue.enqueue("bye", 1)
queue.enqueue("hello", 7)
queue.enqueue("world", 2)

print(queue.dequeue())
print(queue.dequeue())
print(queue.dequeue())

hello
world
bye
